In [1]:
import logging
import sys
sys.path.append('/home/mbewley/Development/benthoz')
logger = logging.getLogger()
handler = logging.FileHandler('notebook_log.txt')
handler.setLevel(logging.DEBUG)
logger.handlers = [handler]

%matplotlib inline
import matplotlib.pyplot as plt
import os
import glob
import pandas as pd
import numpy as np
import imageio
import cv2
from tqdm.notebook import tqdm_notebook as tqdm

import prep.image_fetcher
import prep.label_converter


os.chdir('/home/mbewley/Development/benthoz')
IMAGE_PATH = 'images'
LABELS_PATH = 'labels'

IM_SHAPE = (1024,1360)

if not os.path.exists(IMAGE_PATH):
    os.mkdir(IMAGE_PATH)
    
if not os.path.exists(LABELS_PATH):
    os.mkdir(LABELS_PATH)

IMAGE_LIST_FILE = 'BENTHOZ-2015-imagelist.csv'

# Defining the Hierarchy and Classes
First, we need to look up the class IDs used in the data set, referencing them against class name as per `id_lookups.csv` (also part of the data set paper).
We then need to build the heirarchy to define parent/child relationships.

In [2]:
df_id_lookups = prep.label_converter.build_hierarchy_from_id_lookup(id_lookup_file='idlookups.csv')
df_id_lookups

,name,parsed_name,depth,child_name,ancestor_id_list,descendant_id_list,bit_vector
id,,,,,,,
1,Not Considered,[Not Considered],1,Not Considered,[],[],"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,Biota,[Biota],1,Biota,[],"[3, 13, 30, 32, 39, 73, 81, 88, 102, 118, 165,...","[0, 1, 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, ..."
238,Physical,[Physical],1,Physical,[],"[239, 240, 252, 241, 245, 253, 254, 255, 248, ...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,Worms,"[Biota, Worms]",2,Worms,[2],"[4, 7, 8, 12, 5]","[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
13,Sponges,"[Biota, Sponges]",2,Sponges,[2],"[14, 20, 23, 28, 15, 16, 17, 18, 19, 21, 22, 2...","[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...
161,Cnidaria: Corals: Black & Octocorals: Branchin...,"[Biota, Cnidaria, Corals, Black & Octocorals, ...",7,Arborescent,"[2, 118, 126, 143, 152, 159]",[],"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
149,Cnidaria: Corals: Black & Octocorals: Fan (2D)...,"[Biota, Cnidaria, Corals, Black & Octocorals, ...",7,Simple,"[2, 118, 126, 143, 146, 148]",[],"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
150,Cnidaria: Corals: Black & Octocorals: Fan (2D)...,"[Biota, Cnidaria, Corals, Black & Octocorals, ...",7,Complex,"[2, 118, 126, 143, 146, 148]",[],"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."


In [4]:
# Visualise  the hierarchy automatically, just to sense check it.

## Sparse Points to Image Masks
The next step is to convert the sparse lists of points in (image, row, column, label_id) form, to a 1:1 label mask per image. Initially, we will do this just as a classic multi class problem, with label_id = 0 as the placeholder for "unknown" pixels which should not be considered by the loss function. Later, we will use the hierarchical relationships between the labels to set it up as a multi-label problem.

This means that for an image, the label mask will be a 3D matrix of shape (`len(df_id_lookups)`, `im.shape[0]`, `im.shape[1]`)

In [132]:
DONT_KNOW = -1
NO = 0
YES = 1

def long_labels_to_multilabel_masks(rows, cols, label_ids, im_shape):
    unique_label_ids = list(set(label_ids))
    
    # Stub out the label_mask to -1 for "don't know" unless otherwise specified.
    label_masks = np.ones([len(df_id_lookups), im_shape[0], im_shape[1]], dtype=np.int8) * DONT_KNOW
    for row, col, label_id in zip(*(rows, cols, label_ids)):
        
        # Now we know this particular pixel's value - zero it out, then flip on the relevant bits for the given class to set explicit YES and implicit DONT_KNOWs
        label_masks[:, row, col]
        
        label_masks[:, row, col] = df_id_lookups.loc[label_id, 'bit_vector']
            
    return label_masks

# Data Set Preparation
The first step is to:

1. Acquire the images.
2. Check they have been successfully acquired, by comparing them against the training set.

In [ ]:
prep.image_fetcher.fetch_all_images(IMAGE_LIST_FILE, IMAGE_PATH)

In [2]:
# List number of downloaded images
downloaded_images = [f.strip('.tif') for f in os.listdir(os.path.join(IMAGE_PATH)) if f.endswith('tif')]
downloaded_images
len(downloaded_images)

9436

In [122]:
df_training = pd.read_csv('data_splits/public_labels_train.csv')
df_training.head()

,image_name,row,col,label
0,PR_20081006_232302_383_LC16,82,1255,2
1,PR_20081006_232302_383_LC16,114,586,2
2,PR_20081006_232302_383_LC16,119,647,2
3,PR_20081006_232302_383_LC16,120,1011,2
4,PR_20081006_232302_383_LC16,155,50,241


In [123]:
# row and col slightly exceed image dimensions clip.

df_training.loc[:,'row'] = df_training.row.apply(lambda x: int(min(x, IM_SHAPE[0]-1)))
df_training.loc[:,'col'] = df_training.col.apply(lambda x: min(x, IM_SHAPE[1]-1))
df_training.head()

,image_name,row,col,label
0,PR_20081006_232302_383_LC16,82,1255,2
1,PR_20081006_232302_383_LC16,114,586,2
2,PR_20081006_232302_383_LC16,119,647,2
3,PR_20081006_232302_383_LC16,120,1011,2
4,PR_20081006_232302_383_LC16,155,50,241


In [4]:
training_images = df_training.image_name.unique().tolist()

missing_images = list(set(training_images) - set(downloaded_images) & set(training_images))
missing_image_dates = set([s.split('_')[1] for s in missing_images])
print(f'Missing {len(missing_images)} images on dates: {missing_image_dates}')

Missing 310 images on dates: {'20110416', '20110415'}


In [133]:
# Convert long-form sparse point labels to "don't know" images. Treat this as multi-class for now (single image, with an indep label assigned to each point)

def visualise_label_layer(label_id, im, labels):
    bitvector_ind = df_id_lookups.loc[label_id]['bit_vector_idx']
    print(f'Label {label_id} - displaying layer from bitvector_index {bitvector_ind}')
    labels_single_layer = labels[bitvector_ind]
    
    # Get positive ones and dilate to make visible.
    labels_single_layer = labels[bitvector_ind,:,:]

    label_name = df_id_lookups.loc[label_id]['name']
    plt.figure(figsize=(20,20))
    plt.imshow(im)
    
    labels_pos = ((labels_single_layer == 1)).astype(np.uint8)
    labels_pos_dilated = cv2.dilate(labels_pos, np.ones((9,9),np.uint8))
    plt.imshow(labels_pos_dilated, alpha=labels_pos_dilated, cmap='brg')
    
    labels_neg = ((labels_single_layer == 0)).astype(np.uint8)
    labels_neg_dilated = cv2.dilate(labels_neg, np.ones((9,9),np.uint8))
    plt.imshow(labels_neg_dilated, alpha=labels_neg_dilated, cmap='seismic')

    plt.title(label_name)
    

num_images = len(df_training.image_name.unique())
for i, (image_name, g) in tqdm(enumerate(df_training.groupby('image_name')), total=num_images):

    # Prepare the label mask
    labels = long_labels_to_multilabel_masks(g.row, g.col, g.label, IM_SHAPE)
    
    # Save labels to disk as npz in similar structure to images
    np.savez_compressed(os.path.join(LABELS_PATH, image_name+'.npz'), labels=labels)
    
    
#     im = cv2.imread(os.path.join(IMAGE_PATH, image_name+'.tif'))
#     visualise_label_layer(45, im, labels)
#     break

    